In [11]:
import tensorflow as tf
import mediapipe as mp
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import sklearn
import time

In [12]:
mp_hol = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
mp_face = mp.solutions.face_mesh 

In [13]:
def mp_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [14]:
import os
os.getcwd()


'C:\\SignLangProj'

In [15]:
def draw_landmarks(image, results):
    mp_draw.draw_landmarks(image, results.face_landmarks, mp_face.FACEMESH_TESSELATION,
                           mp_draw.DrawingSpec(color=(85, 110, 10), thickness=1, circle_radius=1),
                           mp_draw.DrawingSpec(color=(85, 256, 121), thickness=1, circle_radius=1)
                          )
    mp_draw.draw_landmarks(image, results.pose_landmarks, mp_hol.POSE_CONNECTIONS, 
                           mp_draw.DrawingSpec(color=(85, 110, 10), thickness=1, circle_radius=4),
                           mp_draw.DrawingSpec(color=(85, 256, 121), thickness=1, circle_radius=2))
    mp_draw.draw_landmarks(image, results.left_hand_landmarks, mp_hol.HAND_CONNECTIONS, mp_draw.DrawingSpec(color=(85, 110, 10), thickness=1, circle_radius=1),
                           mp_draw.DrawingSpec(color=(85, 256, 121), thickness=1, circle_radius=4))
    mp_draw.draw_landmarks(image, results.right_hand_landmarks, mp_hol.HAND_CONNECTIONS, mp_draw.DrawingSpec(color=(85, 110, 10), thickness=1, circle_radius=1),
                           mp_draw.DrawingSpec(color=(85, 256, 121), thickness=1, circle_radius=2))

In [16]:
cap = cv2.VideoCapture(0)

with mp_hol.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mp_detection(frame, holistic)

        draw_landmarks(image, results)
        
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

**Extracting Keypoint Values to make an array from them**

In [17]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks.landmark else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

    return np.concatenate([pose, face, lh, rh])
    

**Creating folders to collect data**

In [18]:
DATA_PATH = os.path.join('MP_Data')

##detecting 3 diff actions
actions = np.array(['hello', 'thanks', 'i love you'])

##30 vids of data
no_seq = 30

#each vid is 30 frames long
seq_len = 30

In [19]:
for action in actions:
    for sequence in range (no_seq):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass